In [21]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [37]:
df = pd.read_csv('data/DEF.csv', sep=',', low_memory=False)

In [38]:
# some craziness to allow line breaks on bubbles
def string_chunks(string_list):
    string_list.sort()
    chunk_size = 2 if len(string_list) > 1 else 1
    new_list =[string_list[i:i+chunk_size] for i in range(0, len(string_list), chunk_size)]
    str_to_return = ','
    for new in new_list:
        if len(new_list) > 1:
            str_to_return += '<br>'
        str_to_return  += ','.join((str(e) for e in new))
        str_to_return += ','
    if len(string_list) > 2:
        str_to_return = str_to_return.replace(',<br>','<br>[',1)
    else:
        str_to_return = str_to_return.replace(',','[',1)
    
    str_to_return = str_to_return[::-1].replace(',', ']',1)[::-1]

    if len(string_list) > 2:
        str_to_return += "<br>"
   
    return str_to_return

In [39]:
kase_contribution_concat = df.groupby(['SE Knowledge Area','RQ3 (Type of contribution)'])['Paper ID'].apply(list).sort_values(ascending=False).reset_index()
kase_contribution_concat['number_of_papers'] = kase_contribution_concat['Paper ID'].apply(lambda x: len(x))
kase_contribution_concat['paper_id_text'] = kase_contribution_concat['Paper ID'].apply(lambda x: string_chunks(x))
kase_contribution_concat.drop(kase_contribution_concat.loc[kase_contribution_concat['RQ3 (Type of contribution)']=='Introductory'].index, inplace=True)
kase_contribution_concat.drop(kase_contribution_concat.loc[kase_contribution_concat['SE Knowledge Area']=='General'].index, inplace=True)

fig = px.scatter(kase_contribution_concat, y="SE Knowledge Area", x="RQ3 (Type of contribution)",
	         size='number_of_papers', 
                 hover_name="RQ3 (Type of contribution)", log_x=False, size_max=75, text="paper_id_text",
                 width=730, height=700)
fig.update_xaxes(type='category') # to print all values, not intervals
fig.update_layout(title = '', 
    plot_bgcolor='rgba(0,0,0,0)')
fig.update_traces(marker_color='rgb(0,102,204)',
                  marker_line_width=1.5)
fig.update_xaxes(categoryorder='category ascending')
fig.update_xaxes(title = '', tickangle=45, gridcolor='#eee')
fig.update_yaxes(title = '', tickangle=45, gridcolor='#eee')
fig.update_yaxes(tickfont = dict(size=14))
fig.update_xaxes(tickfont = dict(size=14))
fig.update_layout(
    font=dict(
        #family="Arial Black",
        size=13,  # Set the font size here
        color="Black"
    )
)
fig.show()
# requires kaleido (https://plotly.com/python/static-image-export/)
fig.write_image("images/bubble-se-ka.pdf", width=695, height=700)